In [50]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib.learn.python.learn.utils.input_fn_utils import InputFnOps

# 1. saving and restoring variables

In [14]:
with tf.variable_scope("v", reuse=tf.AUTO_REUSE):
    v = tf.get_variable("v", shape=[2, 2], dtype=tf.float32)
    add_op = v.assign_add(v + 1)
saver = tf.train.Saver()

In [19]:
with tf.Session() as sess:
    sess.run(v.initializer)
    print add_op.eval()
    save_path = saver.save(sess, "/tmp/tensorflow/model.ckpt")
    print("save session to {}".format(save_path))

[[ 0.1283555  -0.58816934]
 [ 1.13508368  2.24401355]]
save session to /tmp/tensorflow/model.ckpt


In [20]:
with tf.Session() as sess:
    saver.restore(sess, save_path)
    print(v.eval())

INFO:tensorflow:Restoring parameters from /tmp/tensorflow/model.ckpt
[[ 0.1283555  -0.58816934]
 [ 1.13508368  2.24401355]]


# 2. Model save and restoring

In [23]:
# model parameters
graph = tf.Graph()
with graph.as_default():
    W = tf.get_variable("W", [1], tf.float64)
    b = tf.get_variable("b", [1], tf.float64)
    # model input and output
    x = tf.placeholder(tf.float64)
    y = tf.placeholder(tf.float64)
    linear_model = W * x + b
    loss = tf.reduce_sum(tf.square(y - linear_model))
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    train = optimizer.minimize(loss)

In [40]:
import shutil
import numpy as np
export_dir = "/tmp/tensorflow/linear_model"
shutil.rmtree(export_dir)
builder = tf.saved_model.builder.SavedModelBuilder(export_dir)
with tf.Session(graph=graph) as sess:
    x_train = np.array([1., 2., 3., 4.])
    y_train = np.array([0., -1., -2., -3.])
    sess.run(tf.global_variables_initializer())
    sess.run(train, {x: x_train, y: y_train})
    current_W, current_b, loss = sess.run([W, b, loss], {x: x_train, y: y_train})
    print("current loss")
    print loss
    builder.add_meta_graph_and_variables(sess, tags=["linear_training"])
    builder.save()

current loss
0.640973458504
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/tensorflow/linear_model/saved_model.pb


In [41]:
with tf.Session(graph=tf.Graph()) as sess:
    model = tf.saved_model.loader.load(sess, ["linear_training"], export_dir)

INFO:tensorflow:Restoring parameters from /tmp/tensorflow/linear_model/variables/variables


# 3. Using saved model with estimators

In [48]:
base_path = "/Users/lgrcyanny/Codecookies/machine-learning-workspace/tensorflow-wp/hands-on-tensorflow/notebooks"
train_path = base_path + "/datasets/iris/iris_training.csv"
test_path = base_path + "/datasets/iris/iris_test.csv"
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=train_path,
    target_dtype=np.int,
    features_dtype=np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(
    filename=test_path,
    target_dtype=np.int,
    features_dtype=np.float32)

In [49]:
feature_columns = [tf.feature_column.numeric_column("x", shape=4)]
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                        hidden_units=[10, 20, 10],
                                        n_classes=3,
                                        model_dir="/tmp/tensorflow/iris_model_chk")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a1c635810>, '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/tensorflow/iris_model_chk', '_save_summary_steps': 100}


In [42]:
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'x': np.array(training_set.data)},
    y=np.array(training_set.target),
    num_epochs=None,
    shuffle=True
)

In [43]:
classifier.train(input_fn=input_fn, steps=100)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tensorflow/iris_model_chk/model.ckpt.
INFO:tensorflow:loss = 124.245, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tensorflow/iris_model_chk/model.ckpt.
INFO:tensorflow:Loss for final step: 13.4628.


In [32]:
def classifier_serving_input_receiver_fn():
    # save the model
    x = tf.placeholder(dtype=tf.float64, shape=[None, 4], name="x")
    features = {"x": x}
    receiver_tensors = features
    return tf.estimator.export.ServingInputReceiver(features, receiver_tensors)

In [57]:
import os
import shutil
export_dir_base = "/tmp/tensorflow/export/linear_model"
if os.path.exists(export_dir_base):
    shutil.rmtree(export_dir_base)
saved_dir = classifier.export_savedmodel(export_dir_base, 
                                         classifier_serving_input_receiver_fn, 
                                         as_text=True)
print "save_dir", saved_dir

INFO:tensorflow:Restoring parameters from /tmp/tensorflow/iris_model_chk/model.ckpt-100
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /tmp/tensorflow/export/linear_model/temp-1513664616/saved_model.pbtxt
save_dir /tmp/tensorflow/export/linear_model/1513664616
